In [ ]:
import requests
import pandas as pd
import os

In [ ]:
url = "https://api.usaspending.gov"
df_opps = pd.read_xml('./GrantsDBExtract20230910v2.xml')

In [ ]:
cfda_urls = pd.read_csv('./AssistanceListings_GrantsGov_PUBLIC_DAILY_20230910.csv', encoding = "ISO-8859-1")

In [ ]:
cfda_urls.loc[cfda_urls['Program Number'] == 10.001]['URL'][0]

In [ ]:
df_opps['PostDate'] = pd.to_datetime(df_opps['PostDate'], format='%m%d%Y')
df_opps['CloseDate'] = pd.to_datetime(df_opps['CloseDate'], format='%m%d%Y')
df_opps = df_opps[df_opps['CloseDate'] > pd.to_datetime("2023-09-01")]
searchfor = ['RFI']
df_opps = df_opps[~df_opps["OpportunityNumber"].str.contains('|'.join(searchfor))]
df_opps = df_opps[df_opps['CFDANumbers'].notna()]
df_opps = df_opps.astype({"CFDANumbers":'float'})

remove_nones = ["Description", "AgencyName", "AdditionalInformationOnEligibility"]
for column in remove_nones:
    blank_list = []
    for i in df_opps[column]:
        if type(i) == str:
            blank_list.append(i)
        else:
            blank_list.append("No " + column)
    df_opps[column] = blank_list

In [ ]:
popl_list = [
                {
                    "country": 'BGD'},
                {
                    "country": "BOL"},
                {
                    "country": "BDI"},
                {
                    "country": "KHM"},
                {
                    "country": "COD"},
                {
                    "country": "DOM"},
                {
                    "country": "ETH"},
                {
                    "country": "GTM"},
                {
                    "country": "HTI"},
                {
                    "country": "IDN"},
                {
                    "country": "KEN"},
                {
                    "country": "MOZ"},
                {
                    "country": "NIC"},
                {
                    "country": "PER"}, 
                {
                    "country": "PHL"},
                {
                    "country": "RWA"},
                {
                    "country": "SSD"},
                {
                    "country": "UGA"},
                ]

country_names = {
    "BGD" : ["Bangladesh", "Dhaka", "Asia", "Pacific", "Bengali", "Bangladeshi"],
    "BOL" : ["Bolivia", "La Paz", "Sucre", "Latin", "Central America", "South America", "Caribbean", "America", "Bolivian"],
    "BDI" : ["Burundi", "Gitega", "Africa", "Burundian"],
    "KHM" : ["Cambodia","Phnom Penh", "Asia", "Pacific", "Cambodian"],
    "DOM" : ["Dominican Republic", "Santo Domingo", "Latin", "Central America", "South America", "Caribbean", "America", "Dominican"],
    "COD" : ["Congo", "Kinshasa", "Africa", "Congolese"],
    "ETH" : ["Ethiopia","Addis Ababa", "Africa", "Ethiopian"],
    "GTM" : ["Guatemala", "Guatemala City", "Latin", "Central America", "South America", "Caribbean", "America", "Guatemalan"],
    "HTI" : ["Haiti", "Port au Prince", "Latin", "Central America", "South America", "Caribbean", "America", "Haitian"],
    "IDN" : ["Indonesia", "Jakarta", "Asia", "Island", "Pacific", "Indonesian"],
    "KEN" : ["Kenya", "Nairobi", "Africa", "Kenyan"],
    "MOZ" : ["Mozambique", "Maputo", "Africa", "Mozambican"],
    "NIC" : ["Nicaragua", "Managua", "Latin", "Central America", "South America", "Caribbean", "America", "Nicaraguan"],
    "PER" : ["Peru", "Lima", "Latin", "Central America", "South America", "Caribbean", "America", "Peruvian"],
    "PHL" : ["Philippines", "Manila", "Asia", "Island", "Pacific", "Filipino"],
    "RWA" : ["Rwanda", "Kigali", "Africa", "Tutsi", "Hutu", "Rwandan"],
    "SSD" : ["Sudan", "Juba", "Africa", "Sudanese"],
    "UGA" : ["Uganda", "Kampala", "Africa", "Ugandan"]
}

date_list = {
    "FY19" : ["2018-10-01", "2019-09-30"], 
    "FY20" : ["2019-10-01", "2020-09-30"], 
    "FY21" : ["2020-10-01", "2021-09-30"], 
    "FY22" : ["2021-10-01", "2022-09-30"], 
    "FY23" : ["2022-10-01", "2023-09-30"]
    }

countries = open("countries.txt", "r")
countries = countries.read()
countries = countries.split("\n")

In [ ]:
html_body_start = """<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
<style>
.collapsible {
  background-color: #777;
  color: white;
  cursor: pointer;
  padding: 18px;
  width: 100%;
  border: none;
  text-align: left;
  outline: none;
  font-size: 15px;
}

.active, .collapsible:hover {
  background-color: #555;
}

.content {
  padding: 0 18px;
  max-height: 0;
  overflow: hidden;
  transition: max-height 0.2s ease-out;
  background-color: #f1f1f1;
}
</style>
</head>
<body>""" #<div style='display: flex; align-items: center; justify-content: center; flex-direction: column;>'

html_body_end = """<script>
var coll = document.getElementsByClassName("collapsible");
var i;

for (i = 0; i < coll.length; i++) {
  coll[i].addEventListener("click", function() {
    this.classList.toggle("active");
    var content = this.nextElementSibling;
    if (content.style.maxHeight){
      content.style.maxHeight = null;
    } else {
      content.style.maxHeight = content.scrollHeight + "px";
    } 
  });
}
</script>

</body>"""

In [ ]:
for country in popl_list:
    os.mkdir(country_names[country['country']][0])

In [ ]:
final_df = pd.DataFrame()
final_df_cfda = pd.DataFrame()
for country in popl_list:
    print(country)

    # try:
    #     os.mkdir(country_names[country['country']][0])
    #     os.mkdir(country_names[country['country']][0] + "/html")
    # except:
    #     continue 

    country_df = pd.DataFrame()
    endpoint_cfda = "/api/v2/search/spending_by_category/cfda/"
    payload_cfda = {
        "limit": 100,
        "page": 1,
        "filters": {
            "award_type_codes": ["02","03","04","05"],
            "time_period": [{"start_date": "2018-10-01", "end_date": "2023-09-30"}],
            "place_of_performance_locations": [country]
        }
    }
    response_cfda = requests.post(f"{url}{endpoint_cfda}", json=payload_cfda)
    data_cfda = response_cfda.json()
    df_cfda = pd.DataFrame(data_cfda["results"])
    df_cfda = df_cfda[df_cfda.amount >= 250_000]

    cfda_start_html = """<div style="display: flex; align-items: center; justify-content: center; flex-direction: column;">
            <h1>""" + country_names[country['country']][0] + """ USG Breakdown</h1>
            <p style="style="font-style: italic;">USG funding per CFDA category for FY18 through FY23</p>
            <table border="1" class="dataframe">
            <thead>
                <tr style="text-align: center;">
                <th></th>
                <th>CFDA</th>
                <th>Name</th>
                <th>Amount (USD)</th>
                </tr>
            </thead>
            <tbody>"""
    
    for index, row in df_cfda.iterrows():
        sam_url = cfda_urls.loc[cfda_urls['Program Number'] == float(row['code'])]['URL'].item()
        cfda_start_html += "<tr><th>" + str(index) + "</th><td><a href='" + sam_url + "' target='_blank'>" + str(row['code']) + "</a></td><td><a href='#" + str(row['code']) + "'>" + row['name'] + "</a></td><td>" + '${:,.2f}'.format(int(row['amount'])) +"</td></tr>"

    cfda_start_html += "</tbody></table></div><br>"

    # with open("./" + country_names[country['country']][0] + "/html/y.html", "w") as file:
    #     file.write(cfda_start_html)

    # if final_df_cfda.empty:
    #     final_df_cfda = df_cfda
    #     df_cfda.to_csv("./" + country_names[country['country']][0] + "/cfda.csv")
    # else:
    #     final_df_cfda = pd.concat([final_df_cfda, df_cfda], ignore_index = True)

    for key in date_list:
        print(key)
        df_full = True
        page_index = 1
        while df_full:
            print(page_index)
            endpoint_award = "/api/v2/search/spending_by_award/"
            payload_award = {
                "limit": 100,
                "page": page_index,
                "filters": {
                    "award_type_codes": ["02","03","04","05"],
                    "time_period": [{"start_date": date_list[key][0], "end_date": date_list[key][1]}],
                    "place_of_performance_locations": [country],
                    "program_numbers": df_cfda.code.to_list()
                },
                "fields": [
                    "Award ID",
                    "Recipient Name",
                    "Start Date",
                    "End Date",
                    "Award Amount",
                    "Awarding Agency",
                    "Awarding Sub Agency",
                    "Contract Award Type",
                    "Award Type",
                    "Funding Agency",
                    "Funding Sub Agency",
                    "CFDA Number",
                    "Description"
                ]
            }
            response_award = requests.post(f"{url}{endpoint_award}", json=payload_award)
            data_award = response_award.json()
            df_award = pd.DataFrame(data_award["results"])
            if df_award.empty:
                df_full = False
            df_award["FY"] = key
            df_award["Country"] = country['country']
            if country_df.empty:
                country_df = df_award
            else:
                country_df = pd.concat([country_df, df_award], ignore_index = True)
            page_index += 1

    country_df = country_df.astype({"CFDA Number":'float'})
    country_df = country_df.drop_duplicates(subset='Award ID', keep="first")
    country_df['End Date'] = pd.to_datetime(country_df['End Date']).dt.date
    country_df = country_df.sort_values(by='End Date')

    countries_copy = countries.copy()
    for related_names in country_names[country['country']]:
        print(related_names)
        countries_copy.remove(related_names)
    df_opps_copy = df_opps[
        ~df_opps["OpportunityTitle"].str.contains('|'.join(countries_copy)) 
        & ~df_opps["Description"].str.contains('|'.join(countries_copy)) 
        & ~df_opps["AgencyName"].str.contains('|'.join(countries_copy))
        & ~df_opps["AdditionalInformationOnEligibility"].str.contains('|'.join(countries_copy))]
    df_opps_copy = df_opps_copy[(df_opps_copy['CFDANumbers'].isin([float(i) for i in country_df['CFDA Number'].unique()]))]
    # df_opps_copy.to_csv("./" + country_names[country['country']][0] + "/opps.csv")

    # with open("./" + country_names[country['country']][0] + "/html/z.html", "w") as file:
    #     file.write(html_body)

    html_body_final = html_body_start
    for cfda, name, amount in zip(df_cfda['code'], df_cfda['name'], df_cfda['amount']):
        sam_url = cfda_urls.loc[cfda_urls['Program Number'] == float(cfda)]['URL'].item()
        html_body_final += "<hr><a href='" + sam_url + "'><h2 id='"+ str(cfda) + "'>" + str(cfda) + ": " + str(name) + "</h2></a><p>Amount provided from FY218 through FY23 <b>" + '${:,.2f}'.format(int(amount)) + "</b></p>"

        for index, row in df_opps_copy[df_opps_copy['CFDANumbers'] == float(cfda)].iterrows():
            print(index)
            html_body_final += "<button class='collapsible'><b>[Posted in " + row['PostDate'].strftime('%Y') + "]  </b> " + str(row['OpportunityTitle']) + "</button>"
            html_body_final += "<div class='content'><p>" + str(row['Description']) + "<br><br>" + "<a href='https://www.grants.gov/web/grants/view-opportunity.html?oppId=" + str(row['OpportunityID']) + "' target='_blank'>Grants.gov Link</a></p></div>"

    html_body_final += html_body_end

    with open("./" + country_names[country['country']][0] + "/open_opps_report.html", "w") as file:
        file.write(cfda_start_html + html_body_final)

    html_body_final = html_body_start
    for cfda, name, amount in zip(df_cfda['code'], df_cfda['name'], df_cfda['amount']):
        sam_url = cfda_urls.loc[cfda_urls['Program Number'] == float(cfda)]['URL'].item()
        html_body_final += "<hr><a href='" + sam_url + "'><h2 id='"+ str(cfda) + "'>" + str(cfda) + ": " + str(name) + "</h2></a><p>Amount provided from FY218 through FY23 <b>" + '${:,.2f}'.format(int(amount)) + "</b></p>"

        for index, row in country_df[country_df['CFDA Number'] == float(cfda)].iterrows():
            print(index)
            html_body_final += "<button class='collapsible'><b>[Ending in " + str(row['End Date'].year) + "]  </b> " + str(row['Award ID']) + " " + str(row['Recipient Name']) + "</button>"
            html_body_final += "<div class='content'><p>" + '${:,.2f}'.format(int(row['Award Amount'])) + " LOA from " + str(row['Start Date']) + " to " + str(row['End Date']) + "<br>TYPE: " + str(row['Award Type']) + "<br><br>" + str(row['Description']) + "<br><br>" +"<a href='https://www.usaspending.gov/award/" + str(row['generated_internal_id']) + "' target='_blank'>USAspending.gov Link</a></p></div>"

    html_body_final += html_body_end

    with open("./" + country_names[country['country']][0] + "/awards_report.html", "w", encoding="utf-8") as file:
        file.write(cfda_start_html + html_body_final)

#     """
# <p>Collapsible Set:</p>
# <button class="collapsible">Open Section 1</button>
# <div class="content">
#   <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat.</p>
# </div>
# <button class="collapsible">Open Section 2</button>
# <div class="content">
#   <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat.</p>
# </div>
# <button class="collapsible">Open Section 3</button>
# <div class="content">
#   <p>Lorem ipsum dolor sit amet, consectetur adipisicing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat.</p>
# </div>"""
    
    # if final_df.empty:
    #     final_df = country_df
    #     country_df.to_csv("./" + country_names[country['country']][0] + "/awards.csv")
    #     break
    # else:
    #     final_df = pd.concat([final_df, country_df], ignore_index = True)

    # merger = HtmlMerger(input_directory="./" + country_names[country['country']][0] + "/html/", output_file="./" + country_names[country['country']][0] + "/html/merged.html")
    # merger.merge(clean=True)

In [ ]:
country_df

In [ ]:
endpoint_acc = "/api/v2/search/spending_by_category/federal_account/"
payload_acc = {
    "limit": 100,
    "page": 1,
    "filters": {
        "award_type_codes": ["02","03","04","05"],
        "time_period": [{"start_date": "2018-10-01", "end_date": "2023-09-30"}],
        "place_of_performance_locations": [{"country": 'BGD'}],
        "award_ids": ["AID388A000900123"]
    }
}
response_acc = requests.post(f"{url}{endpoint_acc}", json=payload_acc)
data_acc = response_acc.json()
df_acc = pd.DataFrame(data_acc["results"])

In [ ]:
df_acc